In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [9]:
!pip install wandb datasets transformers sacrebleu==1.5.1 -qqq

     |████████████████████████████████| 270 kB 5.2 MB/s 
     |████████████████████████████████| 2.9 MB 37.9 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 1.3 MB 46.3 MB/s 
     |████████████████████████████████| 125 kB 39.2 MB/s 
     |████████████████████████████████| 243 kB 48.1 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 636 kB 45.2 MB/s 
     |████████████████████████████████| 895 kB 35.0 MB/s 
     |████████████████████████████████| 271 kB 45.1 MB/s 
     |████████████████████████████████| 160 kB 43.4 MB/s 


In [13]:
!pip install bert-score -qqq

     |████████████████████████████████| 59 kB 3.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
!nvidia-smi

Mon Oct 11 13:02:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
import datetime
model_checkpoint = 't5-small'
fp16 = True
todaydate = datetime.datetime.now().strftime("%Y-%m-%d")
print(todaydate)
gdir = f'drive/My Drive/Colab Notebooks/{todaydate}/'
model_name=f'sparql-translator-{todaydate}-{model_checkpoint}' + '-fp16' if fp16 else ''
model_path='./models/'+model_name
ds_path= 'lc-quad-wikidata-2021-10-11'
ds_name = 'yazdipour/text-to-sparql-t5-lc-quad-v2'

2021-10-11


In [34]:
!unzip {ds_path}.zip

Archive:  lc-quad-wikidata-2021-10-11.zip
 extracting: lc-quad-wikidata-2021-10-11/dataset_dict.json  
   creating: lc-quad-wikidata-2021-10-11/test/
  inflating: lc-quad-wikidata-2021-10-11/test/dataset.arrow  
  inflating: lc-quad-wikidata-2021-10-11/test/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-11/test/state.json  
   creating: lc-quad-wikidata-2021-10-11/train/
  inflating: lc-quad-wikidata-2021-10-11/train/dataset.arrow  
  inflating: lc-quad-wikidata-2021-10-11/train/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-11/train/state.json  


In [6]:
# Flexible integration for any Python script
import wandb

# 1. Start a W&B run
wandb.init(project='text-to-sparql', entity='shahriar', name= f'{todaydate}-{model_checkpoint}')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
model_name

'sparql-translator-2021-10-11-t5-small-fp16'

In [8]:
!mkdir models

In [10]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk

In [35]:
raw_datasets = load_from_disk(ds_path)

In [11]:
# raw_datasets = load_dataset("yazdipour/text-to-sparql-t5-lc-quad-v2", data_files={"train": "train.csv", "test": "test.csv"})

Using custom data configuration text-to-sparql-t5-lc-quad-v2-bb37c48c93c1fd45


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/text-to-sparql-t5-lc-quad-v2-bb37c48c93c1fd45/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# !pip install tqdm==4.49.0

In [12]:
from tqdm import tqdm

In [ ]:
# Preprocessing

In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [36]:
print(raw_datasets['test']['translation'][0]['sparql'],'\n',
      raw_datasets['test']['translation'][0]['en'],'\n',
      tokenizer(raw_datasets['test']['translation'][0]['en']))

select ?answer where [ wd:nico_yennaris wdt:member_of_sports_team ?X . ?X wdt:color ?answer] 
 who colors of player of nico yennaris ? 
 {'input_ids': [113, 2602, 13, 1959, 13, 3, 2532, 32, 3, 63, 35, 29, 1665, 7, 3, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [37]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry
print('\n',max_input_length, max_target_length)

100%|██████████| 38448/38448 [00:00<00:00, 1111174.96it/s]


 248 323


In [39]:
source_lang = "en"
target_lang = "sparql"
prefix = "translate english to sparql: "

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [40]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [41]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38448
    })
    test: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9612
    })
})

In [ ]:
# Fine-tuning the model

In [42]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [43]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [44]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [52]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]
    return preds, labels

In [45]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

In [48]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
metric = load_metric("sacrebleu")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [49]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    P, R, F1 = scorer.score(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    gen_len = np.mean(prediction_lens)
    
    return {"gen_len":gen_len, 'P':P.mean(), 'R':R.mean(), 'F1':F1.mean(), "bleu-score": result["score"], "bleu-precisions": result["precisions"], "bleu-bp": result["bp"]}

In [ ]:
# metric = load_metric("sacrebleu")

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     # result = {"bleu": result["score"]}

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     # result = {k: round(v, 4) for k, v in result.items()}
#     return result

In [50]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [53]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation, __index_level_0__.
***** Running training *****
  Num examples = 38448
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2403


Epoch,Training Loss,Validation Loss,Gen Len,P,R,F1,Bleu-score,Bleu-precisions,Bleu-bp
1,0.854400,0.600161,19.000000,0.502704,0.059806,0.272079,4.219543,"[83.34756782173942, 63.94946878799091, 50.8666439170187, 43.14391476827958]",0.072151


Saving model checkpoint to sparql-translator-2021-10-11-t5-small-fp16/checkpoint-500
Configuration saved in sparql-translator-2021-10-11-t5-small-fp16/checkpoint-500/config.json
Model weights saved in sparql-translator-2021-10-11-t5-small-fp16/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sparql-translator-2021-10-11-t5-small-fp16/checkpoint-500/tokenizer_config.json
Special tokens file saved in sparql-translator-2021-10-11-t5-small-fp16/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [sparql-translator-2021-10-11-t5-small-fp16/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to sparql-translator-2021-10-11-t5-small-fp16/checkpoint-1000
Configuration saved in sparql-translator-2021-10-11-t5-small-fp16/checkpoint-1000/config.json
Model weights saved in sparql-translator-2021-10-11-t5-small-fp16/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in sparql-translator-2021-10-11-t5-small-fp16/checkpoint-1000/tokenizer_confi

Epoch,Training Loss,Validation Loss


Trainer is attempting to log a value of "[83.34756782173942, 63.94946878799091, 50.8666439170187, 43.14391476827958]" of type <class 'list'> for key "eval/bleu-precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2403, training_loss=0.8568984280513616, metrics={'train_runtime': 2998.9191, 'train_samples_per_second': 12.821, 'train_steps_per_second': 0.801, 'total_flos': 1813926953091072.0, 'train_loss': 0.8568984280513616, 'epoch': 1.0})

In [54]:
trainer.save_model(model_path)

Saving model checkpoint to ./models/sparql-translator-2021-10-11-t5-small-fp16
Configuration saved in ./models/sparql-translator-2021-10-11-t5-small-fp16/config.json
Model weights saved in ./models/sparql-translator-2021-10-11-t5-small-fp16/pytorch_model.bin
tokenizer config file saved in ./models/sparql-translator-2021-10-11-t5-small-fp16/tokenizer_config.json
Special tokens file saved in ./models/sparql-translator-2021-10-11-t5-small-fp16/special_tokens_map.json


In [55]:
!ls -l --block-size=M {model_path}

total 233M
-rw-r--r-- 1 root root   1M Oct 11 15:11 config.json
-rw-r--r-- 1 root root 231M Oct 11 15:11 pytorch_model.bin
-rw-r--r-- 1 root root   1M Oct 11 15:11 special_tokens_map.json
-rw-r--r-- 1 root root   1M Oct 11 15:11 tokenizer_config.json
-rw-r--r-- 1 root root   2M Oct 11 15:11 tokenizer.json
-rw-r--r-- 1 root root   1M Oct 11 15:11 training_args.bin


In [56]:
!mkdir drive/MyDrive/models/{model_name}

In [57]:
!cp {model_path}/* drive/MyDrive/models/{model_name}/

In [58]:
[83.34756782173942, 63.94946878799091, 50.8666439170187, 43.14391476827958].mean()

AttributeError: ignored